In [1]:
import cv2
import numpy as np
from IPython.display import clear_output
import ipywidgets as widgets
from IPython.display import display


In [2]:
# Open the webcam
cap = cv2.VideoCapture(0)

def grab_frame():
    ret, frame = cap.read()
    # Convert the image from BGR to RGB format
    frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    return frame


In [3]:
# Create an image widget
image_widget = widgets.Image(format='jpeg')

# Define a function to display the webcam feed
def display_image(frame):
    # Convert the frame to JPEG
    _, frame = cv2.imencode('.jpeg', frame)
    image_widget.value = frame.tobytes()

# Update the image widget with the current frame
frame = grab_frame()
display_image(frame)

# Display the image widget
display(image_widget)


Image(value=b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x00\x00\x01\x00\x01\x00\x00\xff\xdb\x00C\x00\x02\x01\x0…

In [4]:
import time

# Update the webcam feed every 100ms
try:
    while True:
        frame = grab_frame()
        display_image(frame)
        clear_output(wait=True)
        time.sleep(0.1)
except KeyboardInterrupt:
    # Release the webcam when interrupted
    cap.release()
    print("Webcam feed stopped.")


Webcam feed stopped.


In [8]:
!curl -sL https://nvidia.github.io/libnvidia-container/gpgkey | sudo gpg --dearmor -o /etc/apt/trusted.gpg.d/nvidia.gpg
!curl -sL https://nvidia.github.io/libnvidia-container/$DISTRO/$ARCH/libnvidia-container.list | sudo tee /etc/apt/sources.list.d/libnvidia-container.list


# Unsupported distribution!
# Check https://nvidia.github.io/libnvidia-container


In [10]:
!sudo apt-get update
!sudo apt-get install nvidia-docker2


Hit:1 http://ports.ubuntu.com/ubuntu-ports bionic InRelease
Hit:2 http://packages.ros.org/ros/ubuntu bionic InRelease               
Get:3 http://ports.ubuntu.com/ubuntu-ports bionic-updates InRelease [88.7 kB]
Get:4 http://ports.ubuntu.com/ubuntu-ports bionic-backports InRelease [83.3 kB]
Get:5 http://ports.ubuntu.com/ubuntu-ports bionic-security InRelease [88.7 kB]
Fetched 261 kB in 2s (160 kB/s)                              
Reading package lists... Done
Reading package lists... Done
Building dependency tree       
Reading state information... Done
E: Unable to locate package nvidia-docker2


In [11]:
!sudo apt-get install python3-libnvinfer-dev


Reading package lists... Done
Building dependency tree       
Reading state information... Done
E: Unable to locate package python3-libnvinfer-dev


In [12]:
!pip3 install --extra-index-url https://developer.download.nvidia.com/compute/redist/jp/v46 'tensorflow<2'


Looking in indexes: https://pypi.org/simple, https://developer.download.nvidia.com/compute/redist/jp/v45
     |################################| 230.8 MB 10 kB/s              
  Preparing metadata (setup.py) ... done
     |################################| 301 kB 5.8 MB/s            
  Preparing metadata (setup.py) ... done
     |################################| 126 kB 8.4 MB/s            
     |################################| 5.4 MB 8.4 MB/s            
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     |################################| 57 kB 2.1 MB/s             
     |################################| 50 kB 2.8 MB/s             
     |################################| 3.8 MB 11.8 MB/s            
     |################################| 42 kB 568 kB/s             
     |################################| 503 kB 18.1 MB/s            
     |################################| 75 kB 1.9 MB/s       

  Created wheel for numpy: filename=numpy-1.18.5-cp36-cp36m-linux_aarch64.whl size=12788771 sha256=21bf616a238764348ebae96f02351ba78d02b6894c560f3531188d32071a16d2
  Stored in directory: /root/.cache/pip/wheels/8e/95/fa/4700540e8376bb1def7bf28c60adb71a0ce28d3cf657f6bafa
  Created wheel for termcolor: filename=termcolor-1.1.0-py3-none-any.whl size=4848 sha256=d644d9ebef4d7bbe542e198bce46a762c10a30a92c8dd265b8505a11d84ddc01
  Stored in directory: /root/.cache/pip/wheels/93/2a/eb/e58dbcbc963549ee4f065ff80a59f274cc7210b6eab962acdc
Successfully built numpy termcolor
Failed to build h5py
  Attempting uninstall: numpy
    Found existing installation: numpy 1.13.3
    Uninstalling numpy-1.13.3:
      Successfully uninstalled numpy-1.13.3
    Running setup.py install for h5py ... error
    ERROR: Command errored out with exit status 1:
     command: /usr/bin/python3 -u -c 'import io, os, sys, setuptools, tokenize; sys.argv[0] = '"'"'/tmp/pip-install-4udy3z74/h5py_f1d04a01562540efba2a6b480e0ffad

In [ ]:
import tensorflow as tf
from object_detection.utils import config_util
from object_detection.builders import model_builder
from object_detection.utils import label_map_util
from object_detection.utils import visualization_utils as viz_utils


In [ ]:
import os
import tarfile
import urllib.request

# Download and extract the pre-trained model
MODEL_DATE = '20200711'
MODEL_NAME = 'ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8'
MODEL_TAR_FILENAME = MODEL_NAME + '.tar.gz'
MODELS_DOWNLOAD_BASE = 'http://download.tensorflow.org/models/object_detection/tf2/'
MODEL_DOWNLOAD_LINK = MODELS_DOWNLOAD_BASE + MODEL_DATE + '/' + MODEL_TAR_FILENAME
PATH_TO_MODEL_TAR = os.path.join('models', MODEL_TAR_FILENAME)
PATH_TO_CKPT = os.path.join('models', os.path.join(MODEL_NAME, 'checkpoint/'))
PATH_TO_CFG = os.path.join('models', os.path.join(MODEL_NAME, 'pipeline.config'))

if not os.path.exists(PATH_TO_CKPT):
    os.makedirs(PATH_TO_CKPT)

if not os.path.exists(PATH_TO_CFG):
    os.makedirs(PATH_TO_CFG)

urllib.request.urlretrieve(MODEL_DOWNLOAD_LINK, PATH_TO_MODEL_TAR)
tar_file = tarfile.open(PATH_TO_MODEL_TAR)
tar_file.extractall('models')
tar_file.close()

os.remove(PATH_TO_MODEL_TAR)

# Download the label map
LABEL_FILENAME = 'mscoco_label_map.pbtxt'
LABELS_DOWNLOAD_BASE = 'https://raw.githubusercontent.com/tensorflow/models/master/research/object_detection/data/'
PATH_TO_LABELS = os.path.join('models', MODEL_NAME, LABEL_FILENAME)
urllib.request.urlretrieve(LABELS_DOWNLOAD_BASE + LABEL_FILENAME, PATH_TO_LABELS)


In [ ]:
configs = config_util.get_configs_from_pipeline_file(PATH_TO_CFG)
model_config = configs['model']
detection_model = model_builder.build(model_config=model_config, is_training=False)

# Restore the checkpoint
ckpt = tf.compat.v2.train.Checkpoint(model=detection_model)
ckpt.restore(os.path.join(PATH_TO_CKPT, 'ckpt-0')).expect_partial()

# Load the label map
category_index = label_map_util.create_category_index_from_labelmap(PATH_TO_LABELS, use_display_name=True)


In [ ]:
@tf.function
def detect_fn(image):
    """Detect objects in the image using the detection model."""
    image, shapes = detection_model.preprocess(image)
    prediction_dict = detection_model.predict(image, shapes)
    detections = detection_model.postprocess(prediction_dict, shapes)
    return detections


In [ ]:
def display_image_with_detections(frame):
    # Convert the frame to a tensor
    input_tensor = tf.convert_to_tensor(np.expand_dims(frame, axis=0), dtype=tf.float32)

    # Perform object detection
    detections = detect_fn(input_tensor)

    num_detections = int(detections.pop('num_detections'))
    detections = {key: value[0, :num_detections].numpy()
                 
